In [ ]:
#from google.colab import files
#uploaded = files.upload()

# Instalación de librerias necesarias

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.4.2/spark-3.4.2-bin-hadoop3.tgz
!tar xf spark-3.4.2-bin-hadoop3.tgz
!pip install -q findspark

# Cargamos el entorno

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
import findspark
findspark.init("spark-3.4.2-bin-hadoop3")# SPARK_HOME

### Importamos una serie de librería , importamos Pyspark y creamos una sesión

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import findspark
findspark.init("spark-3.4.2-bin-hadoop3")# SPARK_HOME
from pyspark.sql import SparkSession
ss = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

En el ecosistema de Apache Spark, especialmente cuando se trabaja con PySpark para tareas de Machine Learning, es común utilizar diversas transformaciones y herramientas para preparar los datos antes de entrenar modelos. Los elementos VectorAssembler y StringIndexer son dos transformadores de este tipo, provenientes del módulo pyspark.ml.feature, que juegan roles importantes en la preparación de los datos. A continuación, te explico cada uno:

VectorAssembler
VectorAssembler es una transformación que combina una lista dada de columnas en una única columna vectorial. Es comúnmente utilizado para agrupar varias características en un vector, ya que muchos algoritmos de machine learning en Spark operan sobre este formato de datos. Es decir, convierte las columnas de características individuales en una columna de un vector de características, que luego puede ser utilizado por un estimador o algoritmo de ML.

Por ejemplo, si tienes un dataset con columnas de características como "edad", "altura" y "peso", VectorAssembler te permite combinar estas tres columnas en una única columna que contiene vectores, donde cada vector contiene los valores de "edad", "altura" y "peso" para una fila particular.

StringIndexer
StringIndexer es una transformación que convierte columnas de etiquetas de cadena (texto) en etiquetas de índices numéricos. Esto es especialmente útil cuando las etiquetas o las características categóricas están representadas como cadenas, ya que muchos algoritmos de machine learning prefieren trabajar con datos numéricos. StringIndexer asigna a cada cadena única un índice numérico, comenzando por 0. Esto se hace generalmente antes de pasar los datos a un algoritmo de ML para asegurarse de que las características categóricas se manejen correctamente.

Por ejemplo, si tienes una columna con valores categóricos como "male" y "female", StringIndexer puede convertir estos valores en 0 y 1 (o 1 y 0, dependiendo de la frecuencia de cada categoría) para que puedan ser utilizados en el modelo de aprendizaje.

Ambas transformaciones son esenciales en el proceso de preparación de datos para modelado con PySpark ML, facilitando el manejo de diferentes tipos de datos y asegurando que estén en un formato adecuado para los algoritmos de machine learning.





In [ ]:
irisDF= ss.read.csv("/content/ml_iris.csv", inferSchema=True, header=True, nullValue='?', nanValue='?')

In [ ]:
irisDF.columns

['Id',
 'SepalLengthCm',
 'SepalWidthCm',
 'PetalLengthCm',
 'PetalWidthCm',
 'Species']

In [ ]:
#irisDF = irisDF.select(col('_c0').alias('sepal_length'),
                       #col('_c1').alias('sepal_width'),
                       #col('_c2').alias('petal_length'),
                       #col('_c3').alias('petal_width'),
                       #col('_c4').alias('species'))

In [ ]:
irisDF.take(1)

[Row(Id=1, SepalLengthCm=5.1, SepalWidthCm=3.5, PetalLengthCm=1.4, PetalWidthCm=0.2, Species='Iris-setosa')]

In [ ]:
irisDF.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



El método printSchema() en PySpark se utiliza para imprimir el esquema de un DataFrame, lo cual es muy útil para obtener una rápida comprensión de la estructura de los datos. El "esquema" define las columnas de un DataFrame, incluyendo el nombre de cada columna, su tipo de dato (como StringType, IntegerType, DoubleType, etc.), y si la columna puede contener valores nulos.

Cuando invocas printSchema() en un DataFrame de PySpark, se muestra una descripción jerárquica del esquema en un formato fácil de leer. Por cada columna, se listarán el nombre, el tipo de dato y si es nullable (permite valores nulos) o no. Este método no devuelve un valor, sino que simplemente imprime el esquema al estándar de salida.

In [ ]:
vectorAssembler = VectorAssembler(inputCols=['SepalLengthCm',
                                             'SepalWidthCm',
                                             'PetalLengthCm',
                                             'PetalWidthCm'],
                                  outputCol='features')

In [ ]:
virisDF = vectorAssembler.transform(irisDF)
virisDF.take(1)

[Row(Id=1, SepalLengthCm=5.1, SepalWidthCm=3.5, PetalLengthCm=1.4, PetalWidthCm=0.2, Species='Iris-setosa', features=DenseVector([5.1, 3.5, 1.4, 0.2]))]

Definición: Primero, se define un VectorAssembler especificando las columnas de entrada (las características que quieres combinar) y el nombre de la columna de salida (donde se almacenarán los vectores resultantes). Las columnas de entrada pueden ser de tipos numéricos, booleanos, o vectores; el VectorAssembler los concatenará en un único vector.

Transformación: Cuando invocas transform(irisDF) en el VectorAssembler, este procesa el DataFrame irisDF, tomando los valores de las columnas especificadas en cada fila, combinándolos en un vector, y almacenando el vector en la columna de salida especificada. Este proceso se aplica a todas las filas del DataFrame.

Resultado: El resultado es un nuevo DataFrame que contiene todas las columnas originales de irisDF más la nueva columna agregada por VectorAssembler. Esta columna adicional contendrá los vectores que representan la combinación de las características especificadas para cada fila.

In [ ]:
indexer = StringIndexer(inputCol='Species', outputCol='label')
iVirisDF = indexer.fit(virisDF).transform(virisDF)
iVirisDF

DataFrame[Id: int, SepalLengthCm: double, SepalWidthCm: double, PetalLengthCm: double, PetalWidthCm: double, Species: string, features: vector, label: double]

Estas líneas de código utilizan StringIndexer, que es una transformación de PySpark del módulo pyspark.ml.feature, para convertir cadenas (strings) en índices numéricos. Esta transformación es útil cuando trabajas con características categóricas en formato de texto que necesitas convertir a un formato numérico para utilizar en algoritmos de machine learning que solo pueden manejar datos numéricos

### Naive Bayes Classification

In [ ]:
iVirisDF.take(1)

[Row(Id=1, SepalLengthCm=5.1, SepalWidthCm=3.5, PetalLengthCm=1.4, PetalWidthCm=0.2, Species='Iris-setosa', features=DenseVector([5.1, 3.5, 1.4, 0.2]), label=0.0)]

In [ ]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
splits = iVirisDF.randomSplit([0.6,0.4],1)
trainDF = splits[0]
testDF = splits[1]

In [ ]:
trainDF.count()

98

In [ ]:
testDF.count()

In [ ]:
iVirisDF.count()

150

In [ ]:
nb = NaiveBayes(modelType='multinomial')

In [ ]:
nbModel = nb.fit(trainDF)

In [ ]:
predictionsDF = nbModel.transform(testDF)

In [ ]:
predictionsDF.take(1)

[Row(Id=1, SepalLengthCm=5.1, SepalWidthCm=3.5, PetalLengthCm=1.4, PetalWidthCm=0.2, Species='Iris-setosa', features=DenseVector([5.1, 3.5, 1.4, 0.2]), label=0.0, rawPrediction=DenseVector([-12.111, -13.4138, -14.0841]), probability=DenseVector([0.7088, 0.1926, 0.0985]), prediction=0.0)]

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')

In [ ]:
nbAccuracy = evaluator.evaluate(predictionsDF)
nbAccuracy

0.9038461538461539

## Multilayer Perceptron Classification

In [ ]:
# Mismo set de entrenamiento y de test

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier



1. First layer has the same number of nodes as there are inputs. There are four measures, so the first layer will be four. You can then create list of layers. Set the first element to be four.

2. Last element should have the same number of neurons as there are types of outputs. There are three types of iris species. Last row will be three.

3. You want to have layers in between to help the multi-layer perceptron learn how to classify correctly. Insert two rows of five neurons each. There is going to be a four layer multi-layer perceptron.

4. First layer will have four neurons, the middle two layers will have five neurons each, and then the output layer will have three neurons. One for each kind of iris species.


In [ ]:
layers = [4,5,5,3]

In [ ]:
mlp = MultilayerPerceptronClassifier(layers=layers, seed=1)

El parámetro layers especifica la arquitectura de la red neuronal del perceptrón multicapa, incluyendo el número de nodos en cada capa. Es una lista de enteros, donde cada entero representa el número de neuronas en una capa específica.

El primer elemento de la lista corresponde al número de neuronas en la capa de entrada, y debe coincidir con el número de características (atributos) de tus datos de entrada.
Los elementos intermedios de la lista representan el número de neuronas en las capas ocultas. Puedes tener cualquier número de capas ocultas, y cada una puede tener un número diferente de neuronas.
El último elemento de la lista corresponde al número de neuronas en la capa de salida, que generalmente coincide con el número de clases en un problema de clasificación.
Por ejemplo, si layers = [4, 5, 4, 3], esto define una red neuronal con:

4 neuronas en la capa de entrada (por ejemplo, 4 características en tus datos),
Dos capas ocultas con 5 y 4 neuronas respectivamente,
3 neuronas en la capa de salida (por ejemplo, para un problema de clasificación con 3 clases posibles).
Seed
El parámetro seed se utiliza para inicializar el generador de números aleatorios. Proporcionar un valor de seed (semilla) asegura que los resultados sean reproducibles. En el contexto de los modelos de machine learning, la inicialización aleatoria de pesos en las redes neuronales es un paso común. Al establecer una semilla específica (seed=1 en tu caso), garantizas que cada vez que ejecutes tu código, la inicialización aleatoria de los pesos sea la misma, llevando a resultados consistentes en múltiples ejecuciones.

Esto es útil durante la fase de desarrollo y experimentación, ya que permite comparar directamente el rendimiento de diferentes configuraciones o ajustes en el modelo, sabiendo que cualquier diferencia en el rendimiento se debe a los cambios realizados y no a la variabilidad en la inicialización aleatoria de los pesos.

En resumen, layers define la estructura de la red neuronal, y seed asegura la reproducibilidad de los resultados al inicializar de manera consistente los pesos de la red.

In [ ]:
mlpModel = mlp.fit(trainDF)

In [ ]:
mlpPredictions = mlpModel.transform(testDF)

In [ ]:
mlpEvaluator = MulticlassClassificationEvaluator(metricName='accuracy')

In [ ]:
mlpAccuracy = mlpEvaluator.evaluate(mlpPredictions)

In [ ]:
mlpAccuracy

0.9615384615384616


## Decision Tree Classification


In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(labelCol='label', featuresCol='features')

In [ ]:
dtModel = dt.fit(trainDF)

In [ ]:
dtPredictions = dtModel.transform(testDF)

In [ ]:
dtEvaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')

In [ ]:
dtAccuracy = dtEvaluator.evaluate(dtPredictions)
dtAccuracy

0.9423076923076923

Classification Algorithms Summary

    Naive Bayes - Works well if the attributes in your data set are what is known as independent of each other (they don't tightly correlate with each other)

    Multilayer perceptron - Good choice when you have non-linear relationships between data elements

    Decision Trees - Good choice for classification for many problems and decision trees are good to start with
